In [ ]:
!pip install transformers torch accelerate huggingface-hub huggingface-cli hf-transfer
import torch
import re

In [ ]:
def count_parameters(model):
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad) / 10**9
    print(f"Model Size:{num_params:.3f}B parameters")
    return int(num_params)

In [ ]:
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import os

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# Load meta-llama/Meta-Llama-3-8B model, config and tokenizer
model_name = "meta-llama/Meta-Llama-3-8B"
model = AutoModelForCausalLM.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
count_parameters(model)

In [ ]:
def extract_model_weigths(reference_model, n_layers):
    params = {}
    cur_layer = 0
    
    for name, module in reference_model.named_modules():
        if isinstance(module, torch.nn.Module):
            layer_match = re.match(r'model\.layers\.(\d+)', name)
            if layer_match and int(layer_match.group(1)) >= n_layers:
                break

        if hasattr(module, 'weight') and module.weight is not None:
                params[f'{name}.weight'] = module.weight.data.clone()
        if hasattr(module, 'bias') and module.bias is not None:
            params[f'{name}.bias'] = module.bias.data.clone()
    
    norm_layer = model.model.norm
    if hasattr(norm_layer, 'weight') and norm_layer.weight is not None:
        params['model.norm.weight'] = norm_layer.weight.data.clone()
    if hasattr(norm_layer, 'bias') and norm_layer.bias is not None:
        params['model.norm.bias'] = norm_layer.bias.data.clone()

    lm_head = reference_model.lm_head
    if hasattr(lm_head, 'weight') and lm_head.weight is not None:
        params["lm_head.weight"] = lm_head.weight.data
    if hasattr(lm_head, 'bias') and lm_head.bias is not None:
        params["lm_head.bias"] = lm_head.bias.data

    return params

In [ ]:
target_model_n_layers = 24
pretrained_weights = extract_model_weights(model, target_model_n_layers)
